# Project Corona
## Philosophy and History of Science with Computational Means
Prof Dr. Gerd Graßhoff
### Filter dataframe, NLTP

# Libraries, data

In [22]:
import pandas as pd
import spacy
from spacy import displacy
from spacy.matcher import Matcher
import re

In [23]:
# Import English library
nlp = spacy.load('en_core_web_sm')

In [24]:
dfRem=pd.read_json("dfRemdesivierResearchObjects.json")

In [25]:
len(dfRem)

2180

In [26]:
dfRem.head(2)

,PMID,title,abstract
0,32448345.0,Evaluation of the efficacy and safety of intra...,BACKGROUND: Coronavirus disease 2019 (COVID-19...
1,32370766.0,Role of adjunctive treatment strategies in COV...,The coronavirus disease (COVID-19) pandemic ha...


# Filter

## selecting rows

In [27]:
suche="We show"
def filter(zeile):
    cond=zeile["abstract"].str.contains(suche,na=False)
    return(cond)

In [28]:
df=dfRem[filter]
df

,PMID,title,abstract
341,NaN,In silico detection of SARS-CoV-2 specific B-c...,Abstract Rapid generation of diagnostics is pa...
537,NaN,Structure based drug discovery by virtual scre...,&lt;p&gt;Background&lt;/p&gt;&lt;p&gt;The curr...
649,NaN,Defining the Pandemic at the State Level: Sequ...,"In December of 2019, a novel coronavirus, SARS..."
685,NaN,COVID-19 research in Wikipedia,Wikipedia is one of the main sources of free k...
1542,NaN,SARS-CoV-2 and SARS-CoV differ in their cell t...,SARS-CoV-2 is a novel coronavirus currently ca...


# Pattern maching

In [29]:
a=df.iloc[0].abstract
a

'Abstract Rapid generation of diagnostics is paramount to understand epidemiology and to control the spread of emerging infectious diseases such as COVID-19. Computational methods to predict serodiagnostic epitopes that are specific for the pathogen could help accelerate the development of new diagnostics. A systematic survey of 27 SARS-CoV-2 proteins was conducted to assess whether existing B-cell epitope prediction methods, combined with comprehensive mining of sequence databases and structural data, could predict whether a particular protein would be suitable for serodiagnosis. Nine of the predictions were validated with recombinant SARS-CoV-2 proteins in the ELISA format using plasma and sera from patients with SARS-CoV-2 infection, and a further 11 predictions were compared to the recent literature. Results appeared to be in agreement with 12 of the predictions, in disagreement with 3, while a further 5 were deemed inconclusive. We showed that two of our top five candidates, the N

In [30]:
abstract=nlp(a)

In [31]:
for i,sent in enumerate(abstract.sents):
    print(f" Satz {i}= {sent}")

 Satz 0= Abstract Rapid generation of diagnostics is paramount to understand epidemiology and to control the spread of emerging infectious diseases such as COVID-19.
 Satz 1= Computational methods to predict serodiagnostic epitopes that are specific for the pathogen could help accelerate the development of new diagnostics.
 Satz 2= A systematic survey of 27 SARS-CoV-2 proteins was conducted to assess whether existing B-cell epitope prediction methods, combined with comprehensive mining of sequence databases and structural data, could predict whether a particular protein would be suitable for serodiagnosis.
 Satz 3= Nine of the predictions were validated with recombinant SARS-CoV-2 proteins in the ELISA format using plasma and sera from patients with SARS-CoV-2 infection, and a further 11 predictions were compared to the recent literature.
 Satz 4= Results appeared to be in agreement with 12 of the predictions, in disagreement with 3, while a further 5 were deemed inconclusive.
 Satz 5=

In [32]:
sentences=[s for s in abstract.sents]
s=sentences[5].text
s

'We showed that two of our top five candidates, the N-terminal fragment of the nucleoprotein and the receptor-binding domain of the spike protein, have the highest sensitivity and specificity and signal-to-noise ratio for detecting COVID-19 sera/plasma by ELISA.'

In [33]:
# https://spacy.io/usage/linguistic-features
doc=nlp(s)
for token in doc:
        print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

We -PRON- PRON PRP nsubj Xx True True
showed show VERB VBD ROOT xxxx True False
that that SCONJ IN mark xxxx True True
two two NUM CD nsubj xxx True True
of of ADP IN prep xx True True
our -PRON- DET PRP$ poss xxx True True
top top ADJ JJ amod xxx True True
five five NUM CD nummod xxxx True True
candidates candidate NOUN NNS pobj xxxx True False
, , PUNCT , punct , False False
the the DET DT det xxx True True
N n ADJ JJ compound X True False
- - PUNCT HYPH punct - False False
terminal terminal NOUN NN compound xxxx True False
fragment fragment NOUN NN appos xxxx True False
of of ADP IN prep xx True True
the the DET DT det xxx True True
nucleoprotein nucleoprotein PROPN NNP pobj xxxx True False
and and CCONJ CC cc xxx True True
the the DET DT det xxx True True
receptor receptor NOUN NN npadvmod xxxx True False
- - PUNCT HYPH punct - False False
binding bind VERB VBG amod xxxx True False
domain domain NOUN NN conj xxxx True False
of of ADP IN prep xx True True
the the DET DT det xxx True

In [34]:
displacy.render(doc, style="ent")

# Pattern matching
https://spacy.io/usage/rule-based-matching

https://explosion.ai/demos/matcher?text=We%20showed%20that%20two%20of%20our%20top%20five%20candidates%2C%20the%20N-terminal%20fragment%20of%20the%20nucleoprotein%20and%20the%20receptor-binding%20domain%20of%20the%20spike%20protein%2C%20have%20the%20highest%20sensitivity%20and%20specificity%20and%20signal-to-noise%20ratio%20for%20detecting%20COVID-19%20sera%2Fplasma%20by%20ELISA.&model=en_core_web_sm&pattern=%5B%7B%22id%22%3A0%2C%22attrs%22%3A%5B%7B%22name%22%3A%22LOWER%22%2C%22value%22%3A%22we%22%7D%2C%7B%22name%22%3A%22POS%22%2C%22value%22%3A%22VERB%22%7D%5D%7D%5D

In [35]:
matcher = Matcher(nlp.vocab)

In [36]:
pattern = [{"LOWER":"we"},{'POS': 'VERB'}]
matcher.add("matching",None,pattern)

In [37]:
matches=matcher(doc)

In [38]:
print(matches)

[(1221037237276548748, 0, 2)]


In [39]:
for match_id, start, end in matches:
    span = doc[start:end]  # The matched span
    print(span.text)

We showed


In [40]:
pattern1 = [{"POS":"PRN"},{"LOWER":"showed"}]
matcher.add("m2",None,pattern1)

In [41]:
matches2=matcher(doc)

In [42]:
for match_id, start, end in matches2:
    span = doc[start:end]  # The matched span
    print(span.text)

We showed
